In [1]:
! pip install tensorflow-gpu==2.0.0.alpha0
import tensorflow as tf
print(tf.__version__)

2.0.0-alpha0


### TFRecords
Because a TFRecord file is a sequence of binary strings, its structure must be specified prior to saving so that it can be properly written and subsequently read back. TensorFlow has two structures for this, tf.train.Example and tf.train.SequenceExample. What you have to do is store each sample of your data in one of these structures, then serialize it, and use tf.python_io.TFRecordWriter to save it to disk.

Here, the float array, data, is converted to the binary format and then saved to disc. 
A feature is a dictionary containing the data that is passed to tf.train.Example prior to serialization and saving.

In [0]:
import tensorflow as tf
import numpy as np

data=np.array([10.,11.,12.,13.,14.,15.])

def npy_to_tfrecords(fname,data):
    writer = tf.io.TFRecordWriter(fname)
    feature={}
    feature['data'] = tf.train.Feature(float_list=tf.train.FloatList(value=data))
    example = tf.train.Example(features=tf.train.Features(feature=feature))
    serialized = example.SerializeToString()
    writer.write(serialized)
    writer.close()

npy_to_tfrecords("./myfile.tfrecords",data)

The code to read the record back is as follows. A parse_function function is constructed that decodes the dataset read back from the file. This requires a dictionary (keys_to_features) with the same name and structure as the saved data:

In [0]:
dataset = tf.data.TFRecordDataset("./myfile.tfrecords")

def parse_function(example_proto):
 keys_to_features = {'data':tf.io.FixedLenSequenceFeature([], dtype = tf.float32, allow_missing = True) }
    parsed_features = tf.io.parse_single_example(serialized=example_proto, features=keys_to_features)
    return parsed_features['data']

dataset = dataset.map(parse_function)
iterator = tf.compat.v1.data.make_one_shot_iterator(dataset)
# array is retrieved as one item
item = iterator.get_next()
print(item)
print(item.numpy())
print(item[2].numpy())

A more complex use-case:

In [0]:
filename = './students.tfrecords'
data = {
            'ID': 61553,
            'Name': ['Jones', 'Felicity'],
            'Scores': [45.6, 97.2] 
        }

In [0]:
# Using this, we can construct a tf.train.Example class, again using the Feature() method. Note how we have to encode our string:

ID = tf.train.Feature(int64_list=tf.train.Int64List(value=[data['ID']]))

Name = tf.train.Feature(bytes_list=tf.train.BytesList(value=[n.encode('utf-8') for n in data['Name']]))

Scores = tf.train.Feature(float_list=tf.train.FloatList(value=data['Scores']))

example = tf.train.Example(features=tf.train.Features(feature={'ID': ID, 'Name': Name, 'Scores': Scores }))

In [0]:
# Serializing and writing this record to disc is the same as TFRecord example 1:

writer = tf.io.TFRecordWriter(filename)
writer.write(example.SerializeToString())
writer.close()

In [0]:
# To read this back, we just need to construct our parse_function function to reflect the structure of the record:

dataset = tf.data.TFRecordDataset("./students.tfrecords")

def parse_function(example_proto):
    keys_to_features = {'ID':tf.io.FixedLenFeature([], dtype = tf.int64),
                       'Name':tf.io.VarLenFeature(dtype = tf.string),
                        'Scores':tf.io.VarLenFeature(dtype = tf.float32)
                       }
    parsed_features = tf.io.parse_single_example(serialized=example_proto, features=keys_to_features)
    return parsed_features["ID"], parsed_features["Name"],parsed_features["Scores"]

In [0]:
dataset = dataset.map(parse_function)

iterator = tf.compat.v1.data.make_one_shot_iterator(dataset)
item = iterator.get_next()
# record is retrieved as one item
print(item)

Now we can extract our data from item (note that the string must be decoded (from bytes) where the default for our Python 3 is utf8). Note also that the string and the array of floats are returned as sparse arrays, and to extract them from the record, we use the sparse array value method:

In [0]:
print("ID: ",item[0].numpy())
name = item[1].values.numpy()
name1= name[0].decode()returned
name2 = name[1].decode('utf8')
print("Name:",name1,",",name2)
print("Scores: ",item[2].values.numpy())